# EMPIRICAL STATISTICAL DOWNSCALING (ESD) FOR SEASONAL CLIMATE PREDICTIONS

**DATE HERE**

**names and affiliations here**

**Welcome to the I4C-Hub notebook for the ESD of seasonal climate predictions!**  

This notebook is prepared to provide you a guideline to recap the ESD analysis conducted for the seasonal climate predictions in the scope of I4C project. It is a platform that makes the users enable to reproduce the data generated in the project. It offers **transparent access to a wide range of information and data**, thereby promoting robust scientific investigations and supporting climate change assessment efforts.

This guideline is divided into several sections. We will begin by outlining the overarching aim and motivation behind the IPCC-Hub, clarifying its purpose and significance in the realm of climate research. Subsequently, we delve into the Description of the available material within the IPCC-Hub, providing insights into the diverse datasets and resources at your disposal for climate analysis. Finally, in the section on Data loading and basic data operations, **we walk you through the fundamental steps required to start working with climate data effectively**.


## Required Packages

In [1]:
library(startR)
library(s2dv)
library(easyNCDF)
library(ncdf4)
library(ClimProjDiags)
library(CSIndicators)

Warning message:
“package ‘startR’ was built under R version 4.2.3”
Warning message:
“package ‘s2dv’ was built under R version 4.2.3”

Attaching package: ‘s2dv’


The following object is masked from ‘package:base’:

    Filter


Warning message:
“package ‘easyNCDF’ was built under R version 4.2.3”
Warning message:
“package ‘ClimProjDiags’ was built under R version 4.2.3”

Attaching package: ‘CSIndicators’


The following object is masked from ‘package:ClimProjDiags’:

    Threshold




Load sunset modules

In [2]:
nbdir <- getwd()
setwd("/home/jovyan/sunset")
source("modules/Loading/Loading.R")
source("modules/Downscaling/Downscaling.R")
source("modules/Skill/Skill.R")

Warning message:
“package ‘log4r’ was built under R version 4.2.3”

Attaching package: ‘log4r’


The following object is masked from ‘package:base’:

    debug


Warning message:
“package ‘docopt’ was built under R version 4.2.3”
Warning message:
“package ‘multiApply’ was built under R version 4.2.3”
Warning message:
“package ‘abind’ was built under R version 4.2.3”
Loading required package: maps

Warning message:
“package ‘maps’ was built under R version 4.2.3”
Loading required package: qmap

Loading required package: fitdistrplus

Loading required package: MASS

Warning message:
“package ‘MASS’ was built under R version 4.2.3”
Loading required package: survival

Loading required package: easyVerification

Warning message:
“package ‘easyVerification’ was built under R version 4.2.3”
Loading required package: SpecsVerification


Attaching package: ‘SpecsVerification’


The following object is masked from ‘package:s2dv’:

    Corr



Attaching package: ‘easyVerification’


The following

## Input Parameters
1. **var**: Which variable to be downscaled. Available variables:
   * hurs: Relative humidity
   * prlr: Precipitation
   * sfcWind: Surface wind
   * tas: Temperature
   * tasmax: Maximum temperature
   * tasmin: Minimum temperature
2. **dwn_type**: Which downscaling methods to be applied. Available methods:
   * analogs: Analogs method
   * intbc: Interpolation + bias correction
   * intlr: Interpolation + linear regression
   * logreg: Logistic regression
3. **freq**: Temporal resolution of the data to be downscaled. Available options are "daily_data" or "monthly_day". Usage of the daily_data is only recommended for the analogs. Other than this method, the improvement does not exist or marginal, despite adding a huge amount of computational load.
4. **int_method**: Interpolation method to be used to disaggregate the coarse scale model data to the high observation resolution. Only required when dwn_type is **'intbc'**, **'intlr'** or **'logreg'**. Otherwise, it should be set to **NULL**. Available methods:
   * bic: Bicubic interpolation
   * bil: Bilinear interpolation
   * con: Conservative interpolation
   * con2: Second-order conservative interpolation
   * dis: Inverse distance weighted (IDW) interpolation
   * laf: Largest area fraction interpolation
   * nn: Nearest neighbour interpolation
5. **bc_method**: Bias adjustment or calibration method to be applied on the disaggregated data through the int_method. Only required when dwn_type is **'intbc'**. Otherwise, it should be set to **NULL**. Available methods:
   * bias: Simple bias adjustment
   * crps_min: Minimization of continuous ranked probability score (CPRS)
   * evmos: Variance inflation method
   * mse_min: Minimization of mean squared error (MSE)
   * qm: Quantile mapping method
   * rpc-based: Maximization of ratio of predictable components (RPC)
6. **lr_method**: Linear regression method. Only required when dwn_type is **'intlr'**. Otherwise, it should be set to **NULL**. Available methods:
   * basic: Basic linear regression. This method is applied on the disaggregated data through the int_method.
   * 4nn: Four nearest neighbours. 4nn skips the interpolation step since the method itself inherently downscale the coarse resolution data to the reference data resolution without the need for interpolation.
7. **log_reg_method**: Logistic regression method to be utilized on the disaggregated data through the int_method. Only required when dwn_type is **'logreg'**. Otherwise, it should be set to **NULL**. Available methods:
   * ens_mean: Usage of the ensemble mean for each time step as the predictor
   * ens_mean_sd: Usage of the mean and standard deviation of the ensemble for each time step as the predictor
   * sorted_members: Usage of the ensemble member values as the predictor after sorted them in decreasing order for each time step
8. **nanalogs**: An integer indicating the number of analogs to be searched. Only required when dwn_type is **'analogs'**. Otherwise, it should be set to **NULL**.
9. **target_month**: An integer representing the month to be downscaled (ranging from 1 to 12).
10. **lt**: Lead time ranging from 0 to 3. For example, if the target month is 2 and the lead time is 0, predictions for February obtained from the GCM initiated in February will be downscaled. If the target month is 2 and the lead time is 1, predictions for February obtained from the GCM initiated in January will be downscaled, and so on.
11. **demon**: The demonstrator (i.e., region), over where the downscaling will be carried out. Available options:
    * Bergen
    * Catalonia
    * Paris
    * Prague

In [3]:
var <- "tas"
dwn_type <- "analogs"
freq <- "daily_mean"
int_method <- NULL
bc_method <- NULL
lr_method <- NULL
log_reg_method <- NULL 
nanalogs <- 1
target_month <- 1
lt <- 0
demon <- "Catalonia"

## Loading data
Before loading the model and reference data, the recipe file should be imported. The recipe is a configuration file through which datasets, forecast horizon, time period, downscaling methods and skill metrics to compute are specified. A base recipe file is already prepared. The required parts are edited based on the user inputs provided above:

In [4]:
diffmon <- target_month - lt
smonth <- paste0(sprintf('%02d', ifelse(diffmon > 0, diffmon, 12 + diffmon)), '01')
# Read recipe
recipe_file <- paste0("/home/jovyan/T31-ESD-hub/Notebooks/Recipe_seasonal.yml")
recipe <- prepare_outputs(recipe_file)
recipe$Analysis$Time$sdate <- smonth
recipe$Analysis$Time$ftime_min <- lt+1
recipe$Analysis$Time$ftime_max <- lt+1
recipe$Analysis$Variables$name <- var
recipe$Analysis$Variables$freq <- freq
if (tolower(demon) == "catalonia") {
  recipe$Analysis$Region$latmin <- 39
  recipe$Analysis$Region$latmax <- 44
  recipe$Analysis$Region$lonmin <- -1
  recipe$Analysis$Region$lonmax <- 4
} else if (tolower(demon) == "bergen") {
  recipe$Analysis$Region$latmin <- 58
  recipe$Analysis$Region$latmax <- 63
  recipe$Analysis$Region$lonmin <- 4
  recipe$Analysis$Region$lonmax <- 9
} else if (tolower(demon) == "paris") {
  recipe$Analysis$Region$latmin <- 46.5
  recipe$Analysis$Region$latmax <- 51.5
  recipe$Analysis$Region$lonmin <- -.5
  recipe$Analysis$Region$lonmax <- 4.5
} else if (tolower(demon) == "prague") {
  recipe$Analysis$Region$latmin <- 47.5
  recipe$Analysis$Region$latmax <- 52.5
  recipe$Analysis$Region$lonmin <- 12
  recipe$Analysis$Region$lonmax <- 17
}

[1] "Saving all outputs to:"
[1] "/home/jovyan/work/out-logs/Recipe_seasonal_20240717161920"
INFO  [2024-07-17 16:19:20] SUNSET: SUbseasoNal to decadal climate forecast post-processing and asSEssmenT suite
    Copyright (C) 2024 BSC-CNS
    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
INFO  [2024-07-17 16:19:20] Checking recipe: /home/jovyan/T31-ESD-hub/Notebooks/Recipe_seasonal.yml


Warning message in readLines(file, warn = readLines.warn):
“incomplete final line found on 'conf/archive.yml'”


WARN  [2024-07-17 16:19:20] The element 'fcst_year' is not defined in the recipe. No forecast year will be used.
WARN  [2024-07-17 16:19:20] No Calibration method was specified, raw data verification.
WARN  [2024-07-17 16:19:20] Downscaling 'type' is empty in the recipe, setting it to 'none'.
INFO  [2024-07-17 16:19:20] ##### RECIPE CHECK SUCCESSFULL #####


In [5]:
diffmon <- target_month - lt
smonth <- paste0(sprintf('%02d', ifelse(diffmon > 0, diffmon, 12 + diffmon)), '01')

Now, data can be retrieved via the Loading module of the Sunset package. 

In [6]:
data <- Loading(recipe)
dim(data$hcst$data)
dim(data$obs$data)
#nc <- nc_open("/home/jovyan/IMPETUS4CHANGE/data/BSC/ECMWF-SEAS5/monthly_mean/tas_f6h/tas_20180501.nc")
#nc

INFO  [2024-07-17 16:19:21] fcst_year empty in the recipe, creating empty fcst object...


Warning message in readLines(file, warn = readLines.warn):
“incomplete final line found on 'conf/archive.yml'”
Warning message:
“! Warning: Parameter 'pattern_dims' not specified. Taking the first dimension,
!   'dat' as 'pattern_dims'.”
Warning message:
“! Warning: Could not find any pattern dim with explicit data set descriptions (in
!   the form of list of lists). Taking the first pattern dim, 'dat', as
!   dimension with pattern specifications.”
* Exploring files... This will take a variable amount of time depending
*   on the issued request and the performance of the file server...

Warning message:
“! Warning: Found specified values for dimension 'time' but no 'time_var'
!   requested. "time_var = 'time'" has been automatically added to the
!   Start call.”
Warning message:
“! Warning: Found specified values for dimension 'latitude' but no 'latitude_var'
!   requested. "latitude_var = 'latitude'" has been automatically added
!   to the Start call.”
Warning message:
“! Warning: Fo

INFO  [2024-07-17 16:20:49] ##### DATA LOADING COMPLETED SUCCESSFULLY #####
INFO  [2024-07-17 16:20:49] DATA SUMMARY:
INFO  [2024-07-17 16:20:49] data$hcst months: January
INFO  [2024-07-17 16:20:49] data$hcst range: Jan 01 1993 to Jan 31 2016
INFO  [2024-07-17 16:20:49] data$hcst dimensions:
INFO  [2024-07-17 16:20:49]       dat       var      sday     sweek     syear      time  latitude longitude 
INFO  [2024-07-17 16:20:49]         1         1         1         1        24        31         6         6 
INFO  [2024-07-17 16:20:49]  ensemble 
INFO  [2024-07-17 16:20:49]        25 
INFO  [2024-07-17 16:20:49] Statistical summary of the data in data$hcst:
INFO  [2024-07-17 16:20:49] Variable: tas (units: K)
INFO  [2024-07-17 16:20:49]    Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
INFO  [2024-07-17 16:20:49]   255.2   276.6   280.7   280.4   285.1   292.0 
INFO  [2024-07-17 16:20:49] ---------------------------------------------
INFO  [2024-07-17 16:20:50] DATA SUMMARY:
INFO  [2024-0

dat       var      sday     sweek     syear      time  latitude longitude 
        1         1         1         1        24        31         6         6 
 ensemble 
       25

dat       var      sday     sweek     syear      time  latitude longitude 
        1         1         1         1        24        31       100       100 
 ensemble 
        1

## Downscaling


In [7]:
# compute climatologies
clim_hcst <- Apply(data$hcst$data, target_dims = c('syear','ensemble'), mean, na.rm = T)$output1
clim_obs <- Apply(data$obs$data, target_dims = c('syear','ensemble'), mean, na.rm = T)$output1
# compute anomalies
if (dwn_type != "Intbc") {
  data$hcst$data <- Ano(data = data$hcst$data, clim = clim_hcst)
  data$obs$data <- Ano(data = data$obs$data, clim = clim_obs)
}
# Modify the recipe
recipe$Analysis$Workflow$Downscaling$type           <- dwn_type
recipe$Analysis$Workflow$Downscaling$int_method     <- int_method
recipe$Analysis$Workflow$Downscaling$bc_method      <- bc_method
recipe$Analysis$Workflow$Downscaling$lr_method      <- lr_method
recipe$Analysis$Workflow$Downscaling$log_reg_method <- log_reg_method
recipe$Analysis$Workflow$Downscaling$nanalogs       <- nanalogs

After preparing the recipe accordingly, the data can be downscaled through the Downscaling function.

In [8]:
# Downscale datasets
downscaled_data <- Downscaling(recipe, data)

Warning message in Analogs(exp = exp$data, obs = obs$data, obsL = obsL$data, exp_lats = exp$coords[[lat_dim]], :
“The borders of the downscaling region have not been provided. Assuming the four borders of the downscaling region are defined by the first and last elements of the parameters 'exp_lats' and 'exp_lons'.”
Warning message:
“! Warning: CDORemap: Using CDO version 2.4.0.”
Warning message:
“! Warning: CDORemap: Using CDO version 2.4.0.”


[1] "##### DOWNSCALING COMPLETE #####"


## Skill Calculation

In [9]:
skill_values <- Skill(recipe, downscaled_data)

WARN  [2024-07-17 16:54:54] cross_validation parameter not defined, setting it to FALSE.
INFO  [2024-07-17 17:36:10] ##### SKILL METRIC COMPUTATION COMPLETE #####


In [10]:
dim(skill_values$bss10)

latitude longitude       var      time 
      100       100         1        31